In [1]:
import xarray as xr
import numpy as np
from my_funs import est_trend
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import BallTree, DistanceMetric
from causalimpact import CausalImpact
import pickle
import logging

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
logger = logging.getLogger(__name__)


/data/home/hamiddashti/hamid/nasa_above/greeness/codes/src/my_funs.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
2023-05-29 19:58:55.260495: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-29 19:58:55.306737: I tensorflow/tsl/cuda/cudart_stub.cc:28] C

In [2]:
lai = xr.open_dataarray(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max.nc"
).rename({"latitude": "lat", "longitude": "lon"})
# ndvi_max = xr.open_dataarray(dir+"data/processed_data/landsat/resampled/ndvi_annual_max.nc")
# ndvi_max = ndvi_max.sel(time=slice("1985","2013")).rename({"latitude":"lat","longitude":"lon"})
# ndvi_max = ndvi_max/1e4
# nir = xr.open_dataarray(dir+"data/processed_data/landsat/resampled/nir.nc")
# nir = nir.sel(time=slice("1985","2013"))
swi = xr.open_dataarray(dir + "data/processed_data/swi/swi.nc")
# lai_max["time"] = pd.date_range("1984","2014",freq = "A")
lai = lai.sel(time=slice("1985", "2014"))  # 1984 has many nan values
arr = xr.open_dataarray(dir + "data/arr_id.nc")
percent_cover = xr.open_dataarray(
    dir + "data/processed_data/percent_cover/percent_cover.nc"
)
percent_cover = percent_cover.loc["1985":"2013"]
percent_cover = percent_cover.round(2)
# If a class is 0 change it no nan to prevent false zeros in diff later
percent_cover = percent_cover.where(percent_cover != 0)
percent_cover["lat"] = lai["lat"]
percent_cover["lon"] = lai["lon"]
t = pd.date_range(start="1985", end="2014", freq="A-Dec").year


In [3]:
lc = percent_cover.isel(band=[0, 2, 3])
lc_diff_abs = abs(lc.diff("time"))
changed = (lc_diff_abs > 0.1).any(["time", "band"])

lai_changed = lai.where(changed == True)
lai_notchanged = lai.where(changed == False)
lai_changed.to_netcdf(dir + "working/lai_changed_all.nc")
lai_notchanged.to_netcdf(dir + "working/lai_notchanged_all.nc")
changed.to_netcdf(dir + "working/changed.nc")


In [4]:
lai_changed_stacked = lai_changed.stack(z=["lon", "lat"])
lai_notchanged_stacked = lai_notchanged.stack(z=["lon", "lat"])
changed_year_lai = lai_changed_stacked.isel(time=9)
df = changed_year_lai.to_dataframe()
coords = np.radians(df[["lat", "lon"]])
dist = DistanceMetric.get_metric("haversine")
tree = BallTree(coords, metric=dist)
indices = tree.query_radius(coords, r=0.005)


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


In [13]:
from tqdm import tqdm

data_arr = []
n = len(indices)
for k in tqdm(range(n)):
    # for k in tqdm(np.arange(302607, 302609)):

    data = []
    idx = indices[k]
    # Get the changed LAI values of the centeral pixel
    center_pixel = idx[np.where(idx == k)]
    center_pixel_lai = lai_changed_stacked[:, center_pixel].values.squeeze()

    # continue if the central pixel is nan
    if np.isnan(center_pixel_lai).all():
        continue

    data.append(center_pixel_lai)
    # Go over the neighboring pixels and get the LAI values of unchanged pixels
    for i in range(len(idx)):
        if idx[i] == center_pixel:
            continue
        tmp = lai_notchanged_stacked[:, idx[i]].values.squeeze()
        # skip if the neighboring pixel is all nans
        if np.isnan(tmp).all():
            continue
        data.append(tmp)
    data_arr.append(np.array(data).transpose())


with open(dir + "working/data_arr", "wb") as fp:
    pickle.dump(data_arr, fp)


100%|██████████| 2/2 [00:00<00:00,  6.11it/s]


In [9]:
data_arr


[array([[1.84     , 1.9200001, 2.0310001, ..., 1.7490001, 1.7500001,
         1.7370001],
        [1.8820001, 1.9990001, 1.9020001, ..., 1.932    , 1.789    ,
         1.7900001],
        [2.4780002, 2.2910001, 2.2700002, ..., 2.193    , 2.2080002,
         2.2110002],
        ...,
        [1.96     , 2.3200002, 2.604    , ..., 1.4100001, 1.079    ,
         1.34     ],
        [2.033    , 2.713    , 2.2120001, ..., 1.575    , 1.243    ,
         1.2830001],
        [2.7380002, 3.226    , 2.9440002, ..., 3.025    , 3.0540001,
         3.0700002]], dtype=float32),
 array([[1.6200001, 1.9200001, 2.4740002, ..., 1.8880001,       nan,
         1.7490001],
        [1.2310001, 1.9990001, 2.2470002, ..., 1.332    ,       nan,
         1.932    ],
        [2.197    , 2.2910001, 2.436    , ..., 1.3160001, 1.4070001,
         2.193    ],
        ...,
        [2.049    , 2.3200002, 2.4540002, ..., 1.4120001,       nan,
         1.4100001],
        [2.8000002, 2.713    , 2.035    , ..., 1.036    ,

In [ ]:
var = lai_changed.isel(time=9).values
# var = arr2
latitude = lai_changed["lat"].values
longitude = lai_changed["lon"].values
arr_val = arr.values
df = (
    pd.DataFrame(var, latitude, longitude)
    .rename_axis(index="lat", columns="lon")
    .stack()
    .rename("lai")
    .reset_index()
)

# Prepare data
coords = np.radians(df[["lat", "lon"]])

dist = DistanceMetric.get_metric("haversine")
tree = BallTree(coords, metric=dist)
indices = tree.query_radius(coords, r=0.003)


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


In [ ]:
lai_changed_stacked = lai_changed.stack(z=["lon", "lat"])
changed_year_lai = lai_changed_stacked.isel(time=9)
df2 = changed_year_lai.to_dataframe()
coords = np.radians(df2[["lat", "lon"]])
dist = DistanceMetric.get_metric("haversine")
tree = BallTree(coords, metric=dist)
indices = tree.query_radius(coords, r=0.005)


In [ ]:
df = df.reset_index()
closes_index = ((df["lat"] - 64.971).abs() + (df["lon"] - (-120.423)).abs()).idxmin()
df.loc[closes_index, "index"]


36068

In [ ]:
df


,index,lat,lon,lai
0,0,74.525002,-121.875000,0.758
1,1,74.525002,-121.824997,0.685
2,2,74.525002,-121.675003,0.739
3,3,74.525002,-121.574997,0.714
4,4,74.474998,-122.175003,0.308
...,...,...,...,...
92042,92042,52.924999,-117.274994,1.606
92043,92043,52.924999,-116.625000,2.690
92044,92044,52.924999,-116.474998,2.970
92045,92045,52.875000,-118.074997,2.459


/data/home/hamiddashti/miniconda3/envs/geo/lib/python3.9/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


In [ ]:
event_year = 2001
lc_diff = percent_cover.diff("time")
event = lc_diff.sel(time=event_year)
lcc = (event.where(event > 0)).sum("band")
thresh = 0.5
lcc_dom = lcc.where(lcc > thresh)
changed_pixels = np.isfinite(lcc_dom)

# Index of pixels that have experienced significant LCC
I = np.where(changed_pixels == True)
print(len(I[0]))


In [ ]:
winsize = 3
win_size_half = int(np.floor(winsize / 2))
changed_pixels_roll = (
    changed_pixels.rolling({"lat": winsize, "lon": winsize}, center=True)
    .construct({"lat": "lat_dim", "lon": "lon_dim"})
    .values
)
lai_max_roll = (
    lai_max.rolling({"lat": winsize, "lon": winsize}, center=True)
    .construct({"lat": "lat_dim", "lon": "lon_dim"})
    .values
)

ndvi_max_roll = (
    ndvi_max.rolling({"lat": winsize, "lon": winsize}, center=True)
    .construct({"lat": "lat_dim", "lon": "lon_dim"})
    .values
)
swi_roll = (
    swi.rolling({"lat": winsize, "lon": winsize}, center=True)
    .construct({"lat": "lat_dim", "lon": "lon_dim"})
    .values
)


percent_cover_roll = (
    percent_cover.rolling({"lat": winsize, "lon": winsize}, center=True)
    .construct({"lat": "lat_dim", "lon": "lon_dim"})
    .values
)

lc_diff_roll = (
    lc_diff.rolling({"lat": winsize, "lon": winsize}, center=True)
    .construct({"lat": "lat_dim", "lon": "lon_dim"})
    .values
)


In [ ]:
windows = np.empty((winsize, winsize))
windows[win_size_half, win_size_half] = np.nan
lc_surronding = np.argwhere(np.isfinite(windows))

# loop over pixels that have changed
n = 5
idx = [I[0][n], I[1][n]]
# lc_diff[:,:,idx[0],idx[1]].to_pandas().plot(kind="bar",stacked=True,figsize=(18,10))
t = np.arange(1985, 2014)
t_diff = np.arange(1986, 2014)
event_index = np.where(t_diff == event_year)[0][0]

i = idx[0]
j = idx[1]

percent_cover_roll_tmp = percent_cover_roll[:, :, i, j, :, :]  # (30, 10, 3, 3)
lai_tmp = lai_max_roll[:, i, j, :, :]
lc_diff_roll_tmp = lc_diff_roll[:, :, idx[0], idx[1], :, :]  # (28, 10, 3, 3)
lai_max_roll_tmp = lai_max_roll[:, idx[0], idx[1], :, :]
ndvi_max_roll_tmp = ndvi_max_roll[:, idx[0], idx[1], :, :]
swi_roll_tmp = swi_roll[:, idx[0], idx[1], :, :]


y = lai_tmp[:, win_size_half, win_size_half]

# loop over neighbor pixels
x_ndvi = []
x_lai = []
x_swi = []

for k in range(len(lc_surronding)):
    i_tmp = [lc_surronding[k][0], lc_surronding[k][1]]

    m = 5  # years to look bac for LCC

    s1 = percent_cover_roll_tmp[
        event_index - (m - 1) : event_index + 1, :, i_tmp[0], i_tmp[1]
    ]
    s2 = np.isfinite(s1)
    # Check if the land covers for the past m years before the events are the same
    if (s2 == s2[0]).all():
        # If above is true then look for LCC
        d1 = lc_diff_roll_tmp[
            event_index - (m - 1) : event_index, :, i_tmp[0], i_tmp[1]
        ]
        d1 = np.nan_to_num(d1, nan=0)

        # Check if LC has not changed over the m years before the events
        if (np.abs(d1) < 0.1).all():
            ndvi_tmp = ndvi_max_roll_tmp[:, i_tmp[0], i_tmp[1]]
            lai_tmp = lai_max_roll_tmp[:, i_tmp[0], i_tmp[1]]
            swi_tmp = swi_roll_tmp[:, i_tmp[0], i_tmp[1]]
            if ~np.isnan(ndvi_tmp).any():
                x_ndvi.append(ndvi_tmp)
            if ~np.isnan(lai_tmp).any():
                x_lai.append(lai_tmp)

            if ~np.isnan(swi_tmp).any():
                x_swi.append(swi_tmp)

x_ndvi = np.array(x_ndvi).transpose()
x_lai = np.array(x_lai).transpose()
x_swi = np.array(x_swi).transpose()
# plt.plot(x_swi)
plt.plot(t, x_lai)
plt.plot(t, x_ndvi)
plt.plot(t, y, linewidth=3, color="red")
plt.vlines(2008, ymin=0, ymax=4)

yy = np.transpose(np.array(y, ndmin=2))
data_lai = np.concatenate([yy, x_lai], axis=1)
data_all = np.concatenate([yy, x_ndvi, x_lai], axis=1)

# np.isnan(data).any()

Data_all = pd.DataFrame(
    data=data_all, index=pd.date_range(start="1985", end="2014", freq="A")
)
Data_lai = pd.DataFrame(
    data=data_lai, index=pd.date_range(start="1985", end="2014", freq="A")
)
print(Data_lai.shape)
print(Data_all.shape)
np.isnan(yy).any()


Dealing with missing values


In [ ]:
# pre_period = [0,16]
# post_period = [17,len(t)-1]
pre_period = ["19851231", "20071231"]
post_period = ["20081231", "20131231"]

ci = CausalImpact(
    Data_lai,
    pre_period,
    post_period,
    model_args={
        "fit_method": "hmc",
        #   'standardize': True,
        "prior_level_sd": 0.01,
    },
)
# ci = CausalImpact(Data_lai, pre_period, post_period)
print(ci.summary())
print(ci.summary("report"))
ci.plot()


In [ ]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/WillianFuks/tfcausalimpact/master/tests/fixtures/arma_data.csv"
)[["y", "X"]]
data.iloc[70:, 0] += 5

pre_period = [0, 69]
post_period = [70, 99]

ci = CausalImpact(data, pre_period, post_period)
print(ci.summary())
print(ci.summary(output="report"))
ci.plot()


In [ ]:
from causalimpact.misc import standardize
import tensorflow_probability as tfp

Data = pd.DataFrame(data)
normed_data, _ = standardize(Data.astype(np.float32))

obs_data = normed_data.iloc[:16, 0]
linear_level = tfp.sts.LocalLinearTrend(observed_time_series=obs_data)
linear_reg = tfp.sts.LinearRegression(
    design_matrix=normed_data.iloc[:, 1:].values.reshape(-1, normed_data.shape[1] - 1)
)
model = tfp.sts.Sum([linear_level, linear_reg], observed_time_series=obs_data)
ci = CausalImpact(data, pre_period, post_period, model=model)
print(ci.summary())
print(ci.summary(output="report"))
ci.plot()
